# Uploading Batch Input File
 - batch file 보낼 때 너무 많이 보내지 말고 구분해서 보내기

In [1]:
from getpass import getpass

openai_api_key = getpass("OPENAI_API_KEY")

In [2]:
from openai import OpenAI
import json
import pandas as pd

client = OpenAI(api_key=openai_api_key)

In [3]:
batch_input_file = client.files.create(
    file=open("PreviewQuiz_Elem6_batch.jsonl","rb"),
    purpose="batch"
)

print(batch_input_file)

FileObject(id='file-3WHtm9Bi38NxcAkfNRticX', bytes=492814, created_at=1739458170, filename='PreviewQuiz_Elem6_batch.jsonl', object='file', purpose='batch', status='processed', status_details=None)


# Create the Batch

In [4]:
# 업로드한 배치 입력 파일 ID 저장
batch_input_file_id = batch_input_file.id
print(batch_input_file_id)

file-3WHtm9Bi38NxcAkfNRticX


In [5]:
batch_info = client.batches.create(
    input_file_id = batch_input_file_id,
    endpoint = "/v1/chat/completions", # 챗봇 api
    completion_window = "24h", # 현재는 이 기능만 가능
    metadata = {
        "description": "preview quiz"
    }
)

In [6]:
response_json = json.loads(batch_info.json())
response_output = json.dumps(response_json, ensure_ascii=False, indent = 2)
print(response_output)

{
  "id": "batch_67ae067b1c1881908fe00873eb6295ab",
  "completion_window": "24h",
  "created_at": 1739458171,
  "endpoint": "/v1/chat/completions",
  "input_file_id": "file-3WHtm9Bi38NxcAkfNRticX",
  "object": "batch",
  "status": "validating",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": null,
  "error_file_id": null,
  "errors": null,
  "expired_at": null,
  "expires_at": 1739544571,
  "failed_at": null,
  "finalizing_at": null,
  "in_progress_at": null,
  "metadata": {
    "description": "preview quiz"
  },
  "output_file_id": null,
  "request_counts": {
    "completed": 0,
    "failed": 0,
    "total": 0
  }
}


# Checking the Status of Batch
 - validating : the input file is being validated before the batch can begin
 - failed : the input file has failed the validation process
 - in_progress : the input file was successfully validated and the batch is currently being run
 - finalizing : the batch has completed and the results are being prepared
 - completed : the batch has been completed and the results are ready
 - expired : the batch was not able to be completed within the 24-hour time window
 - cancelling : the batch is being cancelled (may take up to 10 minutes)
 - cancelled : the batch was cancelled

- batch_id = 'batch_67adfc25b4088190b6eb8cea3e4f4a1e' : previewquiz_elem3
- batch_id = 'batch_67adfc5bd6188190b4d44eabb4a400d9' : previewquiz_elem4
- batch_id = 'batch_67adfc70ecbc81908c9b1a8f2e8a8e91' : previewquiz_elem5
- batch_id = 'batch_67adfc8a5c98819090e5599f2d81ceae' : previewquiz_elem6

In [53]:
batch_id = "batch_67adfc25b4088190b6eb8cea3e4f4a1e"
# track_batch_progress(batch_id)
batch = client.batches.retrieve(batch_id)
batch_json = json.loads(batch.json())
batch_json

{'id': 'batch_67adfc25b4088190b6eb8cea3e4f4a1e',
 'completion_window': '24h',
 'created_at': 1739455525,
 'endpoint': '/v1/chat/completions',
 'input_file_id': 'file-JckkJdo6LxK2xMZwcVJXsN',
 'object': 'batch',
 'status': 'completed',
 'cancelled_at': None,
 'cancelling_at': None,
 'completed_at': 1739458160,
 'error_file_id': None,
 'errors': None,
 'expired_at': None,
 'expires_at': 1739541925,
 'failed_at': None,
 'finalizing_at': 1739458140,
 'in_progress_at': 1739455526,
 'metadata': {'description': 'preview quiz'},
 'output_file_id': 'file-JaC1tMvRQUhuTg9ibteBjb',
 'request_counts': {'completed': 240, 'failed': 0, 'total': 240}}

In [8]:
batch_json['completed_at']

1739458160

## 시간 확인

In [43]:
from datetime import datetime
import pytz

kst = pytz.timezone('Asia/Seoul')

# 변환 함수
def convert_timestamp(starttime, endtime):
    st = datetime.utcfromtimestamp(starttime)
    end = datetime.utcfromtimestamp(endtime)
    dt_kst_st = st.replace(tzinfo=pytz.utc).astimezone(kst)
    dt_kst_end = end.replace(tzinfo=pytz.utc).astimezone(kst)

    return f"시작 시간 : {dt_kst_st}, 끝나는 시간 : {dt_kst_end}"

convert_timestamp(starttime = batch_json['created_at'], endtime = batch_json['completed_at'])


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_18092\1843218754.py:8: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  st = datetime.utcfromtimestamp(starttime)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_18092\1843218754.py:9: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  end = datetime.utcfromtimestamp(endtime)


'시작 시간 : 2025-02-13 23:07:06+09:00, 끝나는 시간 : 2025-02-13 23:21:58+09:00'

# Retrieving the Results
 - file-EVCbFfNykgV9HphK2h9wsk

In [54]:
file_response = client.files.content('file-JaC1tMvRQUhuTg9ibteBjb') # output_file_id 이용
print(file_response.text)

{"id": "batch_req_67ae065c4d808190be263a619a94f890", "custom_id": "request-1", "response": {"status_code": 200, "request_id": "194b48cc02e0149ca543cda26d788303", "body": {"id": "chatcmpl-B0U8MWDNtPhIYaL13O77sVOirW1ka", "object": "chat.completion", "created": 1739455534, "model": "o3-mini-2025-01-31", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\n  \"goal\": [\n    \"\ub9cc\ub0a0 \ub54c \uc778\uc0ac\ud558\ub294 \ud45c\ud604\uc744 \uc0ac\uc6a9\ud560 \uc218 \uc788\uc5b4\uc694.\",\n    \"You can greet someone using expressions like Hello and Hi!.\"\n  ],\n  \"preview_quizzes\": [\n    {\n      \"fill_in_blank_quiz\": \"___ there!\",\n      \"korean_translation\": \"\uc800\uae30, \uc548\ub155!\",\n      \"options\": [\n        \"Hello\",\n        \"Hi\",\n        \"Hey\"\n      ],\n      \"answer\": \"Hi\",\n      \"answer_index\": 2\n    },\n    {\n      \"fill_in_blank_quiz\": \"___!\",\n      \"korean_translation\": \"\uc548\ub155!\",\n      \"options\": [\n   

In [61]:
sample_list = file_response.text.split("\n")
sample_list = [item for item in sample_list if item.strip()]
sample_list

['{"id": "batch_req_67ae065c4d808190be263a619a94f890", "custom_id": "request-1", "response": {"status_code": 200, "request_id": "194b48cc02e0149ca543cda26d788303", "body": {"id": "chatcmpl-B0U8MWDNtPhIYaL13O77sVOirW1ka", "object": "chat.completion", "created": 1739455534, "model": "o3-mini-2025-01-31", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\\n  \\"goal\\": [\\n    \\"\\ub9cc\\ub0a0 \\ub54c \\uc778\\uc0ac\\ud558\\ub294 \\ud45c\\ud604\\uc744 \\uc0ac\\uc6a9\\ud560 \\uc218 \\uc788\\uc5b4\\uc694.\\",\\n    \\"You can greet someone using expressions like Hello and Hi!.\\"\\n  ],\\n  \\"preview_quizzes\\": [\\n    {\\n      \\"fill_in_blank_quiz\\": \\"___ there!\\",\\n      \\"korean_translation\\": \\"\\uc800\\uae30, \\uc548\\ub155!\\",\\n      \\"options\\": [\\n        \\"Hello\\",\\n        \\"Hi\\",\\n        \\"Hey\\"\\n      ],\\n      \\"answer\\": \\"Hi\\",\\n      \\"answer_index\\": 2\\n    },\\n    {\\n      \\"fill_in_blank_quiz\\": \\"___!\\",\\

In [62]:
sample_output = []

for sample in sample_list:
    for i in range(len(sample_list)):
        sample_json = json.loads(sample)
        content = sample_json["response"]["body"]["choices"][0]["message"]["content"]
        content_json = json.loads(content)
        sample_output.append(f"{i} 번째")
        sample_output.append(content_json)
        print(json.dumps(content_json, indent=2, ensure_ascii=False))
        break

# print(json.dumps(sample_output))


{
  "goal": [
    "만날 때 인사하는 표현을 사용할 수 있어요.",
    "You can greet someone using expressions like Hello and Hi!."
  ],
  "preview_quizzes": [
    {
      "fill_in_blank_quiz": "___ there!",
      "korean_translation": "저기, 안녕!",
      "options": [
        "Hello",
        "Hi",
        "Hey"
      ],
      "answer": "Hi",
      "answer_index": 2
    },
    {
      "fill_in_blank_quiz": "___!",
      "korean_translation": "안녕!",
      "options": [
        "Hey",
        "Hello",
        "Goodbye"
      ],
      "answer": "Hello",
      "answer_index": 2
    }
  ],
  "sample_script": [
    {
      "sample_script_eng": "A: Hi there! B: Hey, it's been a while. A: Hello! How have you been? B: I've been doing great.",
      "sample_script_kor": "A: 안녕, 저기! B: 야, 오랜만이야. A: 안녕! 잘 지냈어? B: 난 잘 지내고 있어."
    }
  ]
}
{
  "goal": [
    "극존칭 표현을 통해 정중하게 요청하고 긍정적으로 응답할 수 있어요.",
    "You can use honorific expressions to make requests and respond positively."
  ],
  "preview_quizzes": [
    {
      "fill_

# batch 결과

In [63]:
for sample in sample_list:
    sample_json = json.loads(sample)
    content = sample_json["response"]["body"]["choices"][0]["message"]["content"]
    content_json = json.loads(content)
    print(json.dumps(content_json, indent=2, ensure_ascii=False))
    break

{
  "goal": [
    "만날 때 인사하는 표현을 사용할 수 있어요.",
    "You can greet someone using expressions like Hello and Hi!."
  ],
  "preview_quizzes": [
    {
      "fill_in_blank_quiz": "___ there!",
      "korean_translation": "저기, 안녕!",
      "options": [
        "Hello",
        "Hi",
        "Hey"
      ],
      "answer": "Hi",
      "answer_index": 2
    },
    {
      "fill_in_blank_quiz": "___!",
      "korean_translation": "안녕!",
      "options": [
        "Hey",
        "Hello",
        "Goodbye"
      ],
      "answer": "Hello",
      "answer_index": 2
    }
  ],
  "sample_script": [
    {
      "sample_script_eng": "A: Hi there! B: Hey, it's been a while. A: Hello! How have you been? B: I've been doing great.",
      "sample_script_kor": "A: 안녕, 저기! B: 야, 오랜만이야. A: 안녕! 잘 지냈어? B: 난 잘 지내고 있어."
    }
  ]
}


# CSV 파일로 변환

In [64]:
for sample in sample_list:
    sample_json = json.loads(sample)
    content = sample_json["response"]["body"]["choices"][0]["message"]["content"]
    content_json = json.loads(content)
    # print(json.dumps(content_json, indent=2, ensure_ascii=False))
    # break

In [65]:
content = sample_json["response"]["body"]["choices"][0]["message"]["content"]
content_json = json.loads(content)

In [66]:
def JsonToCSV(data:json, file_name:str) -> json:
    data_list = []

    for sample in data:
        # json 변환환
        sample_json = json.loads(sample)
        content = sample_json["response"]["body"]["choices"][0]["message"]["content"]
        content_json = json.loads(content)

        # 학습 목표
        goal = "\n".join(content_json["goal"])

        # 퀴즈 1
        quiz1 = content_json["preview_quizzes"][0]
        quiz1_data = {
            "Quiz 1": quiz1["fill_in_blank_quiz"],
            "Korean 1": quiz1["korean_translation"],
            "Option 1-1": quiz1["options"][0],
            "Option 1-2": quiz1["options"][1],
            "Option 1-3": quiz1["options"][2],
            "Answer 1": quiz1["answer"],
            "Index 1": quiz1["answer_index"]
        }

        # 퀴즈 2
        quiz2 = content_json["preview_quizzes"][1]
        quiz2_data = {
            "Quiz 2": quiz2["fill_in_blank_quiz"],
            "Korean 2": quiz2["korean_translation"],
            "Option 2-1": quiz2["options"][0],
            "Option 2-2": quiz2["options"][1],
            "Option 2-3": quiz2["options"][2],
            "Answer 2": quiz2["answer"],
            "Index 2": quiz2["answer_index"]
        }

        # script
        sample_script_eng = content_json["sample_script"][0]["sample_script_eng"]
        sample_script_kor = content_json["sample_script"][0]["sample_script_kor"]

        # 데이터 만들기
        csv_data = {
            "학습목표": goal,
            **quiz1_data,
            **quiz2_data,
            "샘플 스크립트 (영어)": sample_script_eng,
            "샘플 스크립트 (한국어)": sample_script_kor
        }

        data_list.append(csv_data)

    df = pd.DataFrame(data_list)

    # CSV 저장
    df.to_csv(f"{file_name}.csv", index=False, encoding='utf-8-sig')

    print(f"CSV 변환 완료 파일명: {file_name}.csv")


In [68]:
name = 'PreviewQuiz_Mid3'
JsonToCSV(sample_list, name)

CSV 변환 완료 파일명: PreviewQuiz_Mid3.csv


In [69]:
df = pd.read_csv(f'{name}.csv')
df.head(1)

,학습목표,Quiz 1,Korean 1,Option 1-1,Option 1-2,Option 1-3,Answer 1,Index 1,Quiz 2,Korean 2,Option 2-1,Option 2-2,Option 2-3,Answer 2,Index 2,샘플 스크립트 (영어),샘플 스크립트 (한국어)
0,만날 때 인사하는 표현을 사용할 수 있어요.\nYou can greet someon...,___ there!,"저기, 안녕!",Hello,Hi,Hey,Hi,2,___!,안녕!,Hey,Hello,Goodbye,Hello,2,"A: Hi there! B: Hey, it's been a while. A: Hel...","A: 안녕, 저기! B: 야, 오랜만이야. A: 안녕! 잘 지냈어? B: 난 잘 지..."


# Cancelling a Batch

In [19]:
# cancel_batch = client.batches.cancel("")
# batch_json = json.loads(cancel_batch.json())
# batch_ouput = json.dumps(batch_json, ensure_ascii=False, indent=2)
# print(batch_ouput)